In [0]:
#Averaging the results obtained from the models trained on 3 orthogonal axes

In [0]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
from sklearn.utils import shuffle
import nibabel as nib #reading MR images
import glob
from matplotlib import pyplot as plt
%matplotlib inline
!pip install nilearn
import nilearn as nil


In [0]:
ff = sorted(glob.glob('drive/My Drive/Skull_Stripped_training_input_images/*'))   #For reading a folder
N=64    #resized image dimensions (N, N)

import cv2
images_train = []    #contains the training data sliced over all 3 axes

#Have used nibabel library which has been imported as 'nib' to read the Nifti images 

for f in range(len(ff)):
    a = nib.load(ff[f])
    a = a.get_fdata()
    for k in range(a.shape[0]):                                                                                                   #for k in range(a.shape[0]):
        temp = cv2.resize(a[k,:,:], (N, N), interpolation = cv2.INTER_LINEAR)        #INTER_CUBIC  (type of interpolation)                                     #temp = cv2.resize(a[:,100+k,:], (N, N), interpolation = cv2.INTER_CUBIC) 
        images_train.append(temp)

images_train = np.asarray(images_train)  #converting list to a numpy array (first dimension refers to the #of samples and the second and the third are the 2D image dimension)
images_train = images_train.reshape(-1,N,N,1)  #-1 is generally used for an unknown dimension

m_x = np.max(images_train)    #Normalization over all the training images to get the intensity values between 0 and 1 
mi_x = np.min(images_train)

images_train_normalized_x = (images_train - mi_x) / (m_x - mi_x)

In [0]:
ff = sorted(glob.glob('drive/My Drive/Skull_Stripped_training_input_images/*'))   #For reading a folder
N=64    #resized image dimensions (N, N)

import cv2
images_train = []    #contains the training data sliced over all 3 axes

#Have used nibabel library which has been imported as 'nib' to read the Nifti images 

for f in range(len(ff)):
    a = nib.load(ff[f])
    a = a.get_fdata()   
    for k in range(a.shape[1]):
        temp = cv2.resize(a[:,k,:], (N, N), interpolation = cv2.INTER_LINEAR) 
        images_train.append(temp)
        
images_train = np.asarray(images_train)  #converting list to a numpy array (first dimension refers to the #of samples and the second and the third are the 2D image dimension)
images_train = images_train.reshape(-1,N,N,1)  #-1 is generally used for an unknown dimension

m_y = np.max(images_train)    #Normalization over the ground truth for all the training images to get the intensity values between 0 and 1
mi_y = np.min(images_train)

images_train_normalized_y = (images_train - mi_y) / (m_y - mi_y)        

In [0]:
ff = sorted(glob.glob('drive/My Drive/Skull_Stripped_training_input_images/*'))   #For reading a folder
N=64    #resized image dimensions (N, N)

import cv2
images_train = []    #contains the training data sliced over all 3 axes

#Have used nibabel library which has been imported as 'nib' to read the Nifti images 

for f in range(len(ff)):
    a = nib.load(ff[f])
    a = a.get_fdata()   
    for k in range(a.shape[2]):
        temp = cv2.resize(a[:,:,k], (N, N), interpolation = cv2.INTER_LINEAR) 
        images_train.append(temp)
images_train = np.asarray(images_train)  #converting list to a numpy array (first dimension refers to the #of samples and the second and the third are the 2D image dimension)
images_train = images_train.reshape(-1,N,N,1)  #-1 is generally used for an unknown dimension

m_z = np.max(images_train)    #Normalization over all the training images to get the intensity values between 0 and 1 
mi_z = np.min(images_train)

images_train_normalized_z = (images_train - mi_z) / (m_z - mi_z)

In [0]:
from keras.models import load_model #load the model that has been trained previously on 3 orthogonal axes separately

model_x = load_model('drive/My Drive/Autoencoder_Models/autoencoder_model_xaxis_epoch50.h5')
model_y = load_model('drive/My Drive/Autoencoder_Models/autoencoder_model_yaxis_epoch50.h5')
model_z = load_model('drive/My Drive/Autoencoder_Models/autoencoder_model_zaxis_epoch50.h5')

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [0]:
pred_x = model_x.predict(images_train_normalized_x)
pred_y = model_y.predict(images_train_normalized_y)
pred_z = model_z.predict(images_train_normalized_z)

# Un-normalizing the data
pred_x_unnormalized=(pred_x* (m_x - mi_x) + mi_x)
pred_y_unnormalized=(pred_y* (m_y - mi_y) + mi_y)
pred_z_unnormalized=(pred_z* (m_z - mi_z) + mi_z)

In [0]:
#Reconstructing bias field on 3 orthogonal axes for one brain volume (can be extended over every volume using a for loop)

f=0   #For a single image
temp=np.empty([260,311,260])
reconstructed_bias_x=[] 
for idx in range(260*f,260*(f+1)):
      img = pred_x_unnormalized[idx, :, :,0]
      img_resized = cv2.resize(img, (260, 311), interpolation=cv2.INTER_CUBIC)
      temp[idx%260,:,:]=img_resized
reconstructed_bias_x.append(temp)

f=0   #For a single image
temp=np.empty([260,311,260])
reconstructed_bias_y=[] 
for idx in range(311*f,311*(f+1)):
      img = pred_y_unnormalized[idx, :, :,0]
      img_resized = cv2.resize(img, (260, 260), interpolation=cv2.INTER_CUBIC)
      temp[:,idx%311,:]=img_resized
reconstructed_bias_y.append(temp)

f=0   #For a single image
temp=np.empty([260,311,260])
reconstructed_bias_z=[] 
for idx in range(260*f,260*(f+1)):
      img = pred_z_unnormalized[idx, :, :,0]
      img_resized = cv2.resize(img, (311,260), interpolation=cv2.INTER_CUBIC)
      temp[:,:,idx%260]=img_resized
reconstructed_bias_z.append(temp)

In [0]:
average_reconstructed_bias=[]
#for f in range(len(ff)):
#For a single case
f=0
average_reconstructed_bias.append((reconstructed_bias_x[f] + reconstructed_bias_y[f] + reconstructed_bias_z[f])/3)

In [0]:
len(reconstructed_bias_x)

1

In [0]:
ff = sorted(glob.glob('drive/My Drive/Skull_Stripped_training_input_images/*'))   #For reading a folder

input_images = []
f=0
a = nib.load(ff[f])
a = a.get_fdata()  
input_images.append(a)

In [0]:
#For a single image
f=0
corrected_images=[]
temp=np.empty([260,311,260])
for i in range(260):
  for j in range(311):
    for k in range(260):
      if average_reconstructed_bias[f][i,j,k]==0:
          temp[i,j,k]=input_images[0][i,j,k]
      else:
          temp[i,j,k]=input_images[0][i,j,k]/average_reconstructed_bias[f][i,j,k]
corrected_images.append(temp)

In [0]:
#Corrected MRI images
corrected_img = nil.image.new_img_like(nib.load(ff[f]),corrected_images[0],copy_header=True)
nib.save(corrected_img,'drive/My Drive/Results/average_corrected_mri.nii.gz')

#Estimated bias field
reconstructed_img = nil.image.new_img_like(nib.load(ff[f]),average_reconstructed_bias[0],copy_header=True)
nib.save(reconstructed_img,'drive/My Drive/Results/average_reconstructed_bias.nii.gz')